In [1]:
import random
import pandas as pd
import Covid19_model as CM
import numpy as np
import shutil
import sys
import os.path
import math
import csv
import lhsmdu
import time
from Pyomo_solve import solve_ode

# Set parameter

In [2]:
D = 1e2
n_site = 16
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 2.7e6
length = 50

In [3]:
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
seed = 0
env = CM.Env_model(init_state, deter_para, in_deter_para, seed)
env.set_para_truth(para_truth)

In [5]:
tic = time.time()
random.seed(seed)
action_store = []
observation, state = init_state.copy(), init_state.copy()
state_buffer, new_buffer  = np.zeros([T, 6]), np.zeros([T, 4])
score, ni, ne = 0, 0, 0
done = False
index = [0]
start_time = 6
aa = [1] * 5
for t in range(T):
    p_set = env.para_sampling(100)
    para = p_set[index[0]]
    action_buffer = [] 
    reward_buffer = []
    s = np.array(state)
    ac = []
    if t > start_time:
        for a in range(n_site):
            try:
                aa = solve_ode(s[:,a], np.sum(s[1:-1,a])*budget/np.sum(s[1:-1,:]), para, length)
            except Exception:
                print("pyomo fail")
            ac.append(aa)
        action = np.array(ac).T
        if budget < 0: 
            action = np.zeros((5,n_site))
    else:
        action = np.zeros((5,n_site))
        
    
    next_state, obs, reward, cost, new_i, new_e, ck = env.online_state_estimate(state, observation, 
                                                                                action, p_set, 100, 1)
    #if not done and t > start_time: 
     #   env.space_update(ck, p_set, n_select)
    index = ck.argsort()[0][:1]
    score += reward
    budget -= cost
    ni += new_i
    ne += new_e
    action_store.append(action)
    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'True State Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, score:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]),
                  budget, score))
    print('\r', 'Observe Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]),
                  np.sum(state[4]), np.sum(state[5]))) 
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time() - tic)

 True State Time: 0 | S: 25495888.0, E: 112.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2700000.00, score:-229.27
 Observe Time: 0 | S: 25495888.0, E: 112.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0
 Time: 0 | new_e:52.9, cum_e:52.9, new_i:47.3, cum_i:47.3
 True State Time: 1 | S: 25495799.8, E: 147.8, A: 0.0, I: 47.3, D: 0.0, R: 5.1, Budget: 2700000.00, score:-554.35
 Observe Time: 1 | S: 25495797.0, E: 143.0, A: 0.0, I: 47.3, D: 0.0, R: 0.0
 Time: 1 | new_e:74.3, cum_e:127.2, new_i:65.4, cum_i:112.6
 True State Time: 2 | S: 25495676.0, E: 199.6, A: 0.0, I: 112.6, D: 0.1, R: 11.8, Budget: 2700000.00, score:-1016.39
 Observe Time: 2 | S: 25495678.0, E: 199.0, A: 0.0, I: 112.6, D: 0.1, R: 0.0
 Time: 2 | new_e:104.9, cum_e:232.1, new_i:91.3, cum_i:204.0
 True State Time: 3 | S: 25495501.2, E: 274.0, A: 0.0, I: 203.8, D: 0.2, R: 20.8, Budget: 2700000.00, score:-1674.21
 Observe Time: 3 | S: 25495510.0, E: 273.0, A: 0.0, I: 203.8, D: 0.2, R: 12.0
 Time: 3 | new_e:148.7, cum_e:380.8, new_i:128.5, cum_

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 21 | S: 24442983.1, E: 4.8, A: 370.7, I: 1018.4, D: 5.2, R: 1051617.8, Budget: 1886166.94, score:-12107.41
 Observe Time: 21 | S: 24443099.0, E: 0.0, A: 370.7, I: 1018.4, D: 5.2, R: 1051626.0
 Time: 21 | new_e:42.5, cum_e:2717.7, new_i:68.6, cum_i:3545.4
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
pyomo fail
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        i

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 45 | S: 23294491.1, E: 8.3, A: 31.2, I: 139.2, D: 7.0, R: 2201323.2, Budget: 1099990.89, score:-12958.20
 Observe Time: 45 | S: 23294730.0, E: 0.0, A: 31.2, I: 139.2, D: 7.0, R: 2201338.0
 Time: 45 | new_e:6.5, cum_e:2893.2, new_i:8.6, cum_i:4010.8
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 46 | S: 23255152.0, E: 10.9, A: 29.1, I: 132.4, D: 7.0, R: 2240668.6, Budget: 1070546.90, score:-12979.38
 Observe Time: 46 | S: 23255397.0, E: 2

 True State Time: 64 | S: 22611560.5, E: 8.2, A: 13.0, I: 47.8, D: 7.2, R: 2884363.3, Budget: 625081.18, score:-13253.37
 Observe Time: 64 | S: 22611881.0, E: 0.0, A: 13.0, I: 47.8, D: 7.2, R: 2884383.0
 Time: 64 | new_e:4.3, cum_e:2958.3, new_i:5.1, cum_i:4112.3
 True State Time: 65 | S: 22574169.5, E: 9.5, A: 12.8, I: 47.5, D: 7.2, R: 2921753.4, Budget: 602496.82, score:-13270.70
 Observe Time: 65 | S: 22574495.0, E: 0.0, A: 12.8, I: 47.5, D: 7.2, R: 2921773.0
 Time: 65 | new_e:3.9, cum_e:2962.2, new_i:5.0, cum_i:4117.3
 True State Time: 66 | S: 22540503.8, E: 9.7, A: 13.1, I: 46.6, D: 7.2, R: 2955419.6, Budget: 582135.93, score:-13288.30
 Observe Time: 66 | S: 22540836.0, E: 0.0, A: 13.1, I: 46.6, D: 7.2, R: 2955440.0
 Time: 66 | new_e:4.0, cum_e:2966.2, new_i:5.1, cum_i:4122.4
 True State Time: 67 | S: 22509937.3, E: 10.2, A: 13.1, I: 45.9, D: 7.2, R: 2985986.3, Budget: 561922.71, score:-13306.86
 Observe Time: 67 | S: 22510276.0, E: 0.0, A: 13.1, I: 45.9, D: 7.2, R: 2986007.0
 Tim

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 87 | S: 22049279.3, E: 175.0, A: 46.1, I: 248.7, D: 7.5, R: 3446243.4, Budget: 255357.98, score:-15392.23
 Observe Time: 87 | S: 22050260.0, E: 0.0, A: 46.1, I: 248.7, D: 7.5, R: 3446241.0
 Time: 87 | new_e:76.5, cum_e:3447.3, new_i:74.7, cum_i:4612.0
 True State Time: 88 | S: 22036756.6, E: 215.1, A: 48.7, I: 295.1, D: 7.6, R: 3458676.9, Budget: 238287.82, score:-15768.20
 Observe Time: 88 | S: 22037852.0, E: 2.0, A: 48.7, I: 295.1, D: 7.6, R: 3458667.0
 Time: 88 | new_e:86.3, cum_e:3533.5, new_i:85.8, cum_i:4697.8
 True State Time: 89 | S: 22024664.7, E: 246.0, A: 63.9, I: 343.9, D: 7.6, R: 3470673.8, Budget: 227094.76, score:-16197.40
 Observe Time: 89 | S: 22025901.0, E: 0.0, A: 63.9, I: 343.9, D: 7.6, R: 3470657.0
 Time: 89 | new_e:98.5, cum_e:3632.0, new_i:98.8, cum_i:4796.6


In [6]:
result_state = pd.DataFrame(state_buffer)
result_state.to_csv(r'MPC_high_state_16.csv')
result_new = pd.DataFrame(new_buffer)
result_new.to_csv(r'MPC_high_new_16.csv')

In [10]:
import pickle
pickle.dump(action_store, open("MPC_high_action.pth", 'wb'))

In [8]:
(time.time() - tic) / ( T - start_time + 1)

101.63398829347948

In [9]:
3632.0 + 4796.6

8428.6